# Wczytywanie danych w sparku

Utworzenie środowiska pyspark do obliczeń:

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q dlcdn.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

In [ ]:
import findspark
findspark.init()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pandas as pd
from google.colab import drive
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame as SparkDataFrame
from pyspark.sql.types import IntegerType, FloatType, StringType, StructType
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from itertools import combinations
import copy
import warnings
warnings.filterwarnings('ignore')

Utowrzenie sesji:

In [ ]:
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

Połączenie z dyskiem:

In [ ]:
drive.mount('/content/drive')

Wczytanie danych NASA znajdujących się na dysku w sparku:

In [ ]:
columns = ['lon', 'lat', 'Date', 'Rainf', 'Evap', 'AvgSurfT', 'Albedo','SoilT_10_40cm', 'GVEG', 'PotEvap', 'RootMoist', 'SoilM_100_200cm']

# Utworzenie schematu określającego typ zmiennych
schema = StructType()
for i in columns:
  if i == "Date":
    schema = schema.add(i, IntegerType(), True)
  else:
    schema = schema.add(i, FloatType(), True)

In [ ]:
# Wczytanie zbioru Nasa w sparku
nasa = spark.read.format('csv').option("header", True).schema(schema).load('/content/drive/MyDrive/BigMess/NASA/NASA.csv')

Zanim zaczniemy pisać kwerendy należy jeszcze dodać nasz DataFrame (df) do "przestrzeni nazw tabel" Sparka:

In [ ]:
nasa.createOrReplaceTempView("nasa")

Rozdzielenie kolumny "Date" na kolumny "Year" oraz "Month"

In [ ]:
nasa_ym = spark.sql("""
          SELECT
          CAST(SUBSTRING(CAST(Date AS STRING), 1, 4) AS INT) AS Year,
          CAST(SUBSTRING(CAST(Date AS STRING), 5, 2) AS INT) AS Month,
          n.*
          FROM nasa n
          """)
nasa_ym = nasa_ym.drop("Date")

In [ ]:
nasa_ym.createOrReplaceTempView("nasa_ym")

# Grupowanie k-średnich

In [ ]:
def MinMaxScaling(df: pd.DataFrame, attributes: list) -> pd.DataFrame:
  """
  Funkcja służąca do przeskalowania wybranych atrybutów za pomocą funkcji MinMaxScaler, a następnie stworzenia nowej ramki danych z tylko przeskalowanymi atrybutami.
  Parametry:
  - df (DataFrame): Pandas DataFrame zawierająca co najmniej atrybuty,
  - attributes (str): atrybuty, które będziemy skalować.
  """
  scaled_data = MinMaxScaler().fit_transform(df[attributes])
  scaled_df = pd.DataFrame(scaled_data, columns=attributes)
  return scaled_df

In [ ]:
def do_kmeans_and_return_df_with_cluster_column(df: pd.DataFrame, scaled_df: pd.DataFrame, n_clusters: int, random_state: int) -> pd.DataFrame:
  """
  Funkcja wykonuje grupowanie k-średnich dla n_clusters klastrów oraz tworzy nową kolumnę z predykcjami algorytmu k-średnich w ramce danych df.
  Parametry:
  - df (DataFrame): Pandas DataFrame zawierająca co najmniej te same kolumny co scaled_df,
  - scaled_df (DataFrame): Pandas DataFrame zawierająca przeskalowane kolumny, na podstawie których dokonywane jest grupowanie,
  - n_clusters (int): maksymalna liczba klastrów,
  - random_state (int): ziarno losowości.
  """
  kmeans = KMeans(n_clusters = n_clusters, init='k-means++', random_state=random_state)
  kmeans.fit(scaled_df)
  pred = kmeans.predict(scaled_df)
  df['cluster'] = pred
  return df

## 1. Dane z 01.07.2023

### 1.1. Kolumny: Evap, Rainf, AvgSurfT, Albedo, SoilM_100_200cm, GVEG, PotEvap, RootMoist, SoilT_10_40cm.

Zacznijmy od próby grupowania dla danych z lipca 2023, używając do tego kolumn: Evap, Rainf, AvgSurfT, Albedo, SoilM_100_200cm, GVEG, PotEvap, RootMoist, SoilT_10_40cm.

In [ ]:
SparkDataFrame_2023_7 = spark.sql("""
                        SELECT
                        *
                        FROM nasa_ym WHERE (Year == 2023) and (Month == 7)
                        order by lon, lat, Year, Month
                        """)

In [ ]:
SparkDataFrame_2023_7.show(5)

In [ ]:
df_2023_7 = SparkDataFrame_2023_7.toPandas()

In [ ]:
df_2023_7.describe()

In [ ]:
scaled_df_2023_7 = MinMaxScaling(df_2023_7, ['Evap', 'Rainf', 'AvgSurfT', 'Albedo', 'SoilM_100_200cm', 'GVEG', 'PotEvap', 'RootMoist', 'SoilT_10_40cm'])

In [ ]:
output_df_2023_7 = do_kmeans_and_return_df_with_cluster_column(df_2023_7, scaled_df_2023_7, 2, 123)

### 1.2. Kolumny z PCA

In [ ]:
SparkDataFrame_2023_7_pca = spark.sql("""
                        SELECT
                        *
                        FROM nasa_ym WHERE (Year == 2023) and (Month == 7)
                        order by lon, lat, Year, Month
                        """)

In [ ]:
df_2023_7_pca = SparkDataFrame_2023_7_pca.toPandas()

In [ ]:
# Standaryzacja
standarized_df_2023_7_pca = StandardScaler().fit_transform(df_2023_7_pca.drop(columns=["Month", "lon", "lat", "Year"]))
# PCA
pca = PCA(n_components=2)
standarized_df_2023_7_pca_done = pca.fit_transform(standarized_df_2023_7_pca)

In [ ]:
# Wyjaśniona wariancja
pca.explained_variance_ratio_

In [ ]:
standarized_df_2023_7_pca_done = pd.DataFrame(data = standarized_df_2023_7_pca_done, columns = ['principal component 1', 'principal component 2'])

In [ ]:
output_df_2023_7_pca = do_kmeans_and_return_df_with_cluster_column(df_2023_7_pca, standarized_df_2023_7_pca_done, 2, 123)

## 2. Dane z 01.01.2023

### 2.1 Kolumny: Evap, Rainf, AvgSurfT, Albedo, SoilM_100_200cm, GVEG, PotEvap, RootMoist, SoilT_10_40cm.

In [ ]:
SparkDataFrame_2023_1 = spark.sql("""
                        SELECT
                        *
                        FROM nasa_ym WHERE (Year == 2023) and (Month == 1)
                        order by lon, lat, Year, Month
                        """)

In [ ]:
df_2023_1 = SparkDataFrame_2023_1.toPandas()

In [ ]:
scaled_df_2023_1 = MinMaxScaling(df_2023_1, ['Evap', 'Rainf', 'AvgSurfT', 'Albedo', 'SoilM_100_200cm', 'GVEG', 'PotEvap', 'RootMoist', 'SoilT_10_40cm'])

In [ ]:
output_df_2023_1 = do_kmeans_and_return_df_with_cluster_column(df_2023_1, scaled_df_2023_1, 2, 123)

### 2.2 Kolumny z PCA

In [ ]:
SparkDataFrame_2023_1_pca = spark.sql("""
                        SELECT
                        *
                        FROM nasa_ym WHERE (Year == 2023) and (Month == 1)
                        order by lon, lat, Year, Month
                        """)

In [ ]:
df_2023_1_pca = SparkDataFrame_2023_1_pca.toPandas()

In [ ]:
standarized_df_2023_1_pca = StandardScaler().fit_transform(df_2023_1_pca.drop(columns=["Month", "lon", "lat", "Year"]))

pca = PCA(n_components=2)
standarized_df_2023_1_pca_done = pca.fit_transform(standarized_df_2023_1_pca)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
standarized_df_2023_1_pca_done = pd.DataFrame(data = standarized_df_2023_1_pca_done, columns = ['principal component 1', 'principal component 2'])

In [ ]:
output_df_2023_1_pca = do_kmeans_and_return_df_with_cluster_column(df_2023_1_pca, standarized_df_2023_1_pca_done, 2, 123)

## 3. Dane z 01.07.2000

### 3.1. Kolumny: Evap, Rainf, AvgSurfT, Albedo, SoilM_100_200cm, GVEG, PotEvap, RootMoist, SoilT_10_40cm.

In [ ]:
SparkDataFrame_2000_7 = spark.sql("""
                        SELECT
                        *
                        FROM nasa_ym WHERE (Year == 2000) and (Month == 7)
                        order by lon, lat, Year, Month
                        """)

In [ ]:
df_2000_7 = SparkDataFrame_2000_7.toPandas()

In [ ]:
scaled_df_2000_7 = MinMaxScaling(df_2000_7, ['Evap', 'Rainf', 'AvgSurfT', 'Albedo', 'SoilM_100_200cm', 'GVEG', 'PotEvap', 'RootMoist', 'SoilT_10_40cm'])

In [ ]:
output_df_2000_7 = do_kmeans_and_return_df_with_cluster_column(df_2000_7, scaled_df_2000_7, 2, 123)

### 3.2. Kolumny z PCA

In [ ]:
SparkDataFrame_2000_7_pca = spark.sql("""
                        SELECT
                        *
                        FROM nasa_ym WHERE (Year == 2000) and (Month == 7)
                        order by lon, lat, Year, Month
                        """)

In [ ]:
df_2000_7_pca = SparkDataFrame_2000_7_pca.toPandas()

In [ ]:
# Standaryzacja
standarized_df_2000_7_pca = StandardScaler().fit_transform(df_2000_7_pca.drop(columns=["Month", "lon", "lat", "Year"]))
# PCA
pca = PCA(n_components=3)
standarized_df_2000_7_pca_done = pca.fit_transform(standarized_df_2000_7_pca)

In [ ]:
# Wyjaśniona wariancja
pca.explained_variance_ratio_

In [ ]:
standarized_df_2000_7_pca_done = pd.DataFrame(data = standarized_df_2000_7_pca_done, columns = ['principal component 1', 'principal component 2', 'principal component 3'])

In [ ]:
output_df_2000_7_pca = do_kmeans_and_return_df_with_cluster_column(df_2000_7_pca, standarized_df_2000_7_pca_done, 2, 123)

## 4. Dane z 01.01.2000

### 4.1. Kolumny: Evap, Rainf, AvgSurfT, Albedo, SoilM_100_200cm, GVEG, PotEvap, RootMoist, SoilT_10_40cm.



In [ ]:
SparkDataFrame_2000_1 = spark.sql("""
                        SELECT
                        *
                        FROM nasa_ym WHERE (Year == 2000) and (Month == 1)
                        order by lon, lat, Year, Month
                        """)

In [ ]:
df_2000_1 = SparkDataFrame_2000_1.toPandas()

In [ ]:
scaled_df_2000_1 = MinMaxScaling(df_2000_1, ['Evap', 'Rainf', 'AvgSurfT', 'Albedo', 'SoilM_100_200cm', 'GVEG', 'PotEvap', 'RootMoist', 'SoilT_10_40cm'])

In [ ]:
output_df_2000_1 = do_kmeans_and_return_df_with_cluster_column(df_2000_1, scaled_df_2000_1, 2, 123)

### 4.2. Kolumny z PCA

In [ ]:
SparkDataFrame_2000_1_pca = spark.sql("""
                        SELECT
                        *
                        FROM nasa_ym WHERE (Year == 2000) and (Month == 1)
                        order by lon, lat, Year, Month
                        """)

In [ ]:
df_2000_1_pca = SparkDataFrame_2000_1_pca.toPandas()

In [ ]:
standarized_df_2000_1_pca = StandardScaler().fit_transform(df_2000_1_pca.drop(columns=["Month", "lon", "lat", "Year"]))

pca = PCA(n_components=2)
standarized_df_2000_1_pca_done = pca.fit_transform(standarized_df_2000_1_pca)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
standarized_df_2000_1_pca_done = pd.DataFrame(data = standarized_df_2000_1_pca_done, columns = ['principal component 1', 'principal component 2'])

In [ ]:
output_df_2000_1_pca = do_kmeans_and_return_df_with_cluster_column(df_2000_1_pca, standarized_df_2000_1_pca_done, 2, 123)

## 5. Dane z 01.07.1979

### 5.1. Kolumny: Evap, Rainf, AvgSurfT, Albedo, SoilM_100_200cm, GVEG, PotEvap, RootMoist, SoilT_10_40cm.

In [ ]:
SparkDataFrame_1979_7 = spark.sql("""
                        SELECT
                        *
                        FROM nasa_ym WHERE (Year == 1979) and (Month == 7)
                        order by lon, lat, Year, Month
                        """)

In [ ]:
df_1979_7 = SparkDataFrame_1979_7.toPandas()

In [ ]:
scaled_df_1979_7 = MinMaxScaling(df_1979_7, ['Evap', 'Rainf', 'AvgSurfT', 'Albedo', 'SoilM_100_200cm', 'GVEG', 'PotEvap', 'RootMoist', 'SoilT_10_40cm'])

In [ ]:
output_df_1979_7 = do_kmeans_and_return_df_with_cluster_column(df_1979_7, scaled_df_1979_7, 2, 123)

### 5.2. Kolumny z PCA

In [ ]:
SparkDataFrame_1979_7_pca = spark.sql("""
                        SELECT
                        *
                        FROM nasa_ym WHERE (Year == 1979) and (Month == 7)
                        order by lon, lat, Year, Month
                        """)

In [ ]:
df_1979_7_pca = SparkDataFrame_1979_7_pca.toPandas()

In [ ]:
# Standaryzacja
standarized_df_1979_7_pca = StandardScaler().fit_transform(df_1979_7_pca.drop(columns=["Month", "lon", "lat", "Year"]))
# PCA
pca = PCA(n_components=3)
standarized_df_1979_7_pca_done = pca.fit_transform(standarized_df_1979_7_pca)

In [ ]:
# Wyjaśniona wariancja
pca.explained_variance_ratio_

In [ ]:
standarized_df_1979_7_pca_done = pd.DataFrame(data = standarized_df_1979_7_pca_done, columns = ['principal component 1', 'principal component 2', 'principal component 3'])

In [ ]:
output_df_1979_7_pca = do_kmeans_and_return_df_with_cluster_column(df_1979_7_pca, standarized_df_1979_7_pca_done, 2, 123)

## 6. Dane z 01.01.1979

### 6.1. Kolumny: Evap, Rainf, AvgSurfT, Albedo, SoilM_100_200cm, GVEG, PotEvap, RootMoist, SoilT_10_40cm.



In [ ]:
SparkDataFrame_1979_1 = spark.sql("""
                        SELECT
                        *
                        FROM nasa_ym WHERE (Year == 1979) and (Month == 1)
                        order by lon, lat, Year, Month
                        """)

In [ ]:
df_1979_1 = SparkDataFrame_1979_1.toPandas()

In [ ]:
scaled_df_1979_1 = MinMaxScaling(df_1979_1, ['Evap', 'Rainf', 'AvgSurfT', 'Albedo', 'SoilM_100_200cm', 'GVEG', 'PotEvap', 'RootMoist', 'SoilT_10_40cm'])

In [ ]:
output_df_1979_1 = do_kmeans_and_return_df_with_cluster_column(df_1979_1, scaled_df_1979_1, 2, 123)

### 6.2. Kolumny z PCA

In [ ]:
SparkDataFrame_1979_1_pca = spark.sql("""
                        SELECT
                        *
                        FROM nasa_ym WHERE (Year == 1979) and (Month == 1)
                        order by lon, lat, Year, Month
                        """)

In [ ]:
df_1979_1_pca = SparkDataFrame_1979_1_pca.toPandas()

In [ ]:
standarized_df_1979_1_pca = StandardScaler().fit_transform(df_1979_1_pca.drop(columns=["Month", "lon", "lat", "Year"]))

pca = PCA(n_components=2)
standarized_df_1979_1_pca_done = pca.fit_transform(standarized_df_1979_1_pca)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
standarized_df_1979_1_pca_done = pd.DataFrame(data = standarized_df_1979_1_pca_done, columns = ['principal component 1', 'principal component 2'])

In [ ]:
output_df_1979_1_pca = do_kmeans_and_return_df_with_cluster_column(df_1979_1_pca, standarized_df_1979_1_pca_done, 2, 123)

# Graficzne przedstawienie klastrów

## Wczytanie pakietów i tworzenie funkcji

In [ ]:
!pip install datashader

In [ ]:
!pip install holoviews hvplot colorcet

In [ ]:
!pip install geoviews

In [ ]:
import numpy as np
import matplotlib as mpl

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf
import colorcet as cc
import holoviews as hv
from holoviews.operation.datashader import datashade
import geoviews as gv
import geoviews.tile_sources as gts
from holoviews import opts
from IPython.display import IFrame
from IPython.core.display import display
from bokeh.plotting import show, output_notebook

In [ ]:
'''
Funkcja jako argumenty bierze listę wartości określających granice przedziałów liczbowych, które
będą określać jak dla rozważanego parametru mają zmieniać się kolory punktów, których lista stanowi
drugi argument funkcji.
'''
def get_colormap(values: list, colors_palette: list, name = 'custom'):
    values = np.sort(np.array(values))
    values = np.interp(values, (values.min(), values.max()), (0, 1))
    cmap = mpl.colors.LinearSegmentedColormap.from_list(name, list(zip(values, colors_palette)))
    return cmap

In [ ]:
def plot_map(df: pd.DataFrame, parameter_name: str, colormap: mpl.colors.LinearSegmentedColormap, title: str,
             point_size: int = 8, width: int = 800, height: int = 500, alpha: float = 1,
             bgcolor: str = 'white', colorbar_verbose: bool = True):

    gdf = gv.Points(df, ['lon', 'lat'], [parameter_name]) # obiekt zawierający punkty
    tiles = gts.OSM # wybór mapy tła, w tym wypadku OpenStreetMap

    # łączenie mapy tła z punktami i ustawienie wybranych parametrów wizualizacji
    map_with_points = tiles * gdf.opts(
        title=title,
        color=parameter_name,
        cmap=colormap,
        size=point_size,
        width=width,
        height=height,
        colorbar=colorbar_verbose,
        toolbar='above',
        tools=['hover', 'wheel_zoom', 'reset'],
        alpha=alpha, # przezroczystość
        bgcolor=bgcolor
    )

    return hv.render(map_with_points)

## Przedstawienie klastrów na mapach

In [ ]:
display(IFrame("https://www.google.com/maps/embed?pb=!1m14!1m12!1m3!1d13982681.959428234!2d-98.66341902257437!3d38.39997874427714!2m3!1f0!2f0!3f0!3m2!1i1024!2i768!4f13.1!5e1!3m2!1spl!2spl!4v1703000232420!5m2!1spl!2spl", '800px', '500px'))

In [ ]:
colormap_cluster = get_colormap([0, max(output_df_2023_7.cluster.values)], ['yellow', 'darkgreen'])
output_notebook()
show(plot_map(df=output_df_2023_7, parameter_name='cluster', colormap=colormap_cluster, title="K-średnich dla danych z 01.07.2023 (wszystkie kolumny)", alpha=0.5))

In [ ]:
colormap_cluster = get_colormap([0, max(output_df_2023_7_pca.cluster.values)], ['yellow', 'darkgreen'])
output_notebook()
show(plot_map(df=output_df_2023_7_pca, parameter_name='cluster', colormap=colormap_cluster, title="K-średnich dla danych z 01.07.2023 (PCA)", alpha=0.5))

In [ ]:
colormap_cluster = get_colormap([0, max(output_df_2023_1.cluster.values)], ['yellow', 'darkgreen'])
output_notebook()
show(plot_map(df=output_df_2023_1, parameter_name='cluster', colormap=colormap_cluster, title="K-średnich dla danych z 01.01.2023 (wszystkie kolumny)", alpha=0.5))

In [ ]:
colormap_cluster = get_colormap([0, max(output_df_2023_1_pca.cluster.values)], ['yellow', 'darkgreen'])
output_notebook()
show(plot_map(df=output_df_2023_1_pca, parameter_name='cluster', colormap=colormap_cluster, title="K-średnich dla danych z 01.01.2023 (PCA)", alpha=0.5))

In [ ]:
colormap_cluster = get_colormap([0, max(output_df_2000_7.cluster.values)], ['yellow', 'darkgreen'])
output_notebook()
show(plot_map(df=output_df_2000_7, parameter_name='cluster', colormap=colormap_cluster, title="K-średnich dla danych z 01.07.2000 (wszystkie kolumny)", alpha=0.5))

In [ ]:
colormap_cluster = get_colormap([0, max(output_df_2000_7_pca.cluster.values)], ['yellow', 'darkgreen'])
output_notebook()
show(plot_map(df=output_df_2000_7_pca, parameter_name='cluster', colormap=colormap_cluster, title="K-średnich dla danych z 01.07.2000 (PCA)", alpha=0.5))

In [ ]:
colormap_cluster = get_colormap([0, max(output_df_2000_1.cluster.values)], ['yellow', 'darkgreen'])
output_notebook()
show(plot_map(df=output_df_2000_1, parameter_name='cluster', colormap=colormap_cluster, title="K-średnich dla danych z 01.01.2000 (wszystkie kolumny)", alpha=0.5))

In [ ]:
colormap_cluster = get_colormap([0, max(output_df_2000_1_pca.cluster.values)], ['yellow', 'darkgreen'])
output_notebook()
show(plot_map(df=output_df_2000_1_pca, parameter_name='cluster', colormap=colormap_cluster, title="K-średnich dla danych z 01.01.2000 (PCA)", alpha=0.5))

In [ ]:
colormap_cluster = get_colormap([0, max(output_df_1979_7.cluster.values)], ['yellow', 'darkgreen'])
output_notebook()
show(plot_map(df=output_df_1979_7, parameter_name='cluster', colormap=colormap_cluster, title="K-średnich dla danych z 01.07.1979 (wszystkie kolumny)", alpha=0.5))

In [ ]:
colormap_cluster = get_colormap([0, max(output_df_1979_7_pca.cluster.values)], ['yellow', 'darkgreen'])
output_notebook()
show(plot_map(df=output_df_1979_7_pca, parameter_name='cluster', colormap=colormap_cluster, title="K-średnich dla danych z 01.07.1979 (PCA)", alpha=0.5))

In [ ]:
colormap_cluster = get_colormap([0, max(output_df_1979_1.cluster.values)], ['yellow', 'darkgreen'])
output_notebook()
show(plot_map(df=output_df_1979_1, parameter_name='cluster', colormap=colormap_cluster, title="K-średnich dla danych z 01.01.1979 (wszystkie kolumny)", alpha=0.5))

In [ ]:
colormap_cluster = get_colormap([0, max(output_df_1979_1_pca.cluster.values)], ['yellow', 'darkgreen'])
output_notebook()
show(plot_map(df=output_df_1979_1_pca, parameter_name='cluster', colormap=colormap_cluster, title="K-średnich dla danych z 01.07.1979 (PCA)", alpha=0.5))

In [ ]:
output_df_1979_7.head()

In [ ]:
output_df_2000_7.head()

In [ ]:
output_df_2023_7.head()

# Porównanie punktów z 01.07.2023, 01.07.2000 i 01.07.1979.

Oznaczenie:


*   **0** - pustynia,
*   **1** - niepustynia.



In [ ]:
merged_to_2023_7 = copy.deepcopy(output_df_2023_7)
merged_to_2023_7["cluster_2000"] = output_df_2000_7["cluster"].apply(lambda x: 0 if x==1 else 1)
merged_to_2023_7["cluster_1979"] = output_df_1979_7["cluster"]
merged_to_2023_7.head(10)

Sprawdźmy, które punkty były zakwalifikowane jako niepustynne w lipcu 2000, a stały się pustynią w lipcu 2023.

In [ ]:
df_from_not_desert_to_desert = merged_to_2023_7.loc[(merged_to_2023_7["cluster"]==0) & (merged_to_2023_7["cluster_2000"]==1)].dropna()
df_from_not_desert_to_desert

In [ ]:
colormap_cluster = get_colormap([0, 1], ['yellow', 'darkgreen'])
output_notebook()
show(plot_map(df=df_from_not_desert_to_desert, parameter_name='cluster', colormap=colormap_cluster, title="Warunek: niepustynia w 01.07.2000 i pustynia w 01.07.2023", alpha=0.5, colorbar_verbose=False))

Sprawdźmy, które punkty były zakwalifikowane jako niepustynne w lipcu 1979, a stały się pustynią w lipcu 2023.

In [ ]:
df_from_not_desert_to_desert = merged_to_2023_7.loc[(merged_to_2023_7["cluster"]==0) & (merged_to_2023_7["cluster_1979"]==1)].dropna()
df_from_not_desert_to_desert

In [ ]:
colormap_cluster = get_colormap([0, 1], ['yellow', 'darkgreen'])
output_notebook()
show(plot_map(df=df_from_not_desert_to_desert, parameter_name='cluster', colormap=colormap_cluster, title="Warunek: niepustynia w 01.07.1979 i pustynia w 01.07.2023", alpha=0.5, colorbar_verbose=False))

Sprawdźmy, jak wyglądają punkty w lipcu 2023, które były zakwalifikowane jako niepustynne w lipcu 1979, a stały się pustynią w lipcu 2000.

In [ ]:
df_from_not_desert_to_desert = merged_to_2023_7.loc[(merged_to_2023_7["cluster_2000"]==0) & (merged_to_2023_7["cluster_1979"]==1)].dropna()
df_from_not_desert_to_desert

In [ ]:
colormap_cluster = get_colormap([0, 1], ['yellow', 'darkgreen'])
output_notebook()
show(plot_map(df=df_from_not_desert_to_desert, parameter_name='cluster', colormap=colormap_cluster, title="Warunek: niepustynia w 01.07.1979 i pustynia w 01.07.2000", alpha=0.5))



*   **0** - pustynia w 01.07.2023,
*   **1** - niepustynia w 01.07.2023.



Widzimy, że niektóre punkty były zakwalifikowane jako niepustynne w lipcu 1997, następnie algorytm zakwalifikował je jako pustynne w lipcu 2000 po czym ponownie w lipcu 2023 zostały uznane za niepustynne.

# Porównanie wyników grupowania z datasetem anotowanym.

Porównanie wyników grupowania metodą k-średnich z wykorzystaniem wszystkich kolumn dla danych z 01.07.2023 z danymi anotowanymi.

In [ ]:
NASA_sample_an = pd.read_csv('/content/drive/MyDrive/BigMess/NASA/NASA_an.csv',sep=';')

In [ ]:
NASA_sample_an.head()

Ponieważ grupowaliśmy dane na dwa klastry, potraktujmy step jako pustynię.

In [ ]:
NASA_sample_an['pustynia_i_step'] = NASA_sample_an.pustynia + NASA_sample_an.step

In [ ]:
NASA_sample_an.head(8)

Zamiana oznaczeń, ponieważ są one odwrotne do tych w zbiorze *merged_to_2023_7*.

In [ ]:
NASA_sample_an["pustynia_i_step"] = NASA_sample_an["pustynia_i_step"].apply(lambda x: 0 if x==1 else 1)

In [ ]:
result = merged_to_2023_7.merge(NASA_sample_an, left_on=['lon','lat'], right_on = ['lon','lat'], how='inner')
result

In [ ]:
colormap_cluster = get_colormap([0, 1], ['yellow', 'darkgreen'])
output_notebook()
show(plot_map(df=result, parameter_name='pustynia_i_step', colormap=colormap_cluster, title="Zanotowane punkty: 0 - pustynia, 1 - niepustynia", alpha=0.5))

In [ ]:
positive = len(result.loc[result.cluster == result.pustynia_i_step])
accuracy = str(round(positive/len(NASA_sample_an)*100,2))
print("Accuracy na poziomie",accuracy+"%.")

Gdybyśmy jednak potraktowali step jako teren niepustynny...

In [ ]:
positive = len(result.loc[result.cluster != result.pustynia])
accuracy = str(round(positive/len(NASA_sample_an)*100,2))
print("Accuracy na poziomie",accuracy+"%.")

Oznacza to, że algorytm potraktował większą część **stepu** jako **pustynię**.

In [ ]:
merged_to_2023_7_pca = copy.deepcopy(output_df_2023_7_pca)
merged_to_2023_7_pca["cluster_2000"] = output_df_2000_7_pca["cluster"].apply(lambda x: 0 if x==1 else 1)
merged_to_2023_7_pca["cluster_1979"] = output_df_1979_7_pca["cluster"]
merged_to_2023_7_pca.head(10)

Porównanie wyników grupowania metodą k-średnich z wykorzystaniem PCA dla danych z 01.07.2023 z danymi anotowanymi.

In [ ]:
result_pca = merged_to_2023_7_pca.merge(NASA_sample_an, left_on=['lon','lat'], right_on = ['lon','lat'], how='inner')
result_pca

In [ ]:
positive = len(result_pca.loc[result_pca.cluster == result_pca.pustynia_i_step])
accuracy = str(round(positive/len(NASA_sample_an)*100,2))
print("Accuracy na poziomie",accuracy+"%.")

Gdybyśmy jednak potraktowali step jako teren niepustynny...

In [ ]:
positive = len(result_pca.loc[result_pca.cluster != result_pca.pustynia])
accuracy = str(round(positive/len(NASA_sample_an)*100,2))
print("Accuracy na poziomie",accuracy+"%.")

# Podsumowanie dotychczasowych wyników



1.   Grupowanie metodą k-średnich wykorzystując dane z okresów zimowych, przy aktualnie wybranych kolumnach, nie przynosi więszkych efektów.
2.   Porównując wyniki algorytmu dla danych z 01.07.2023, 01.07.2000 i 01.07.1979 można zauważyć, że niektóre punkty zmieniły swój stan z niepustynii na pustynię. Są również punkty, które na przestrzeni lat zmieniały swój stan więcej niż jeden raz (punkty te mogą posłużyć jako początek szczegółowej analizy zmian w wartościach wybranych wskaźników, które przyczynniły się do pustynnienia).
3.   W ostatniej części, porównując wyniki z datasetem anotowanym i traktując step jako pustynię, otrzymujemy accuracy na poziomie **89%**. Jeśli uznalibyśmy step jako teren niepustynny, accuracy spada do **77,8%**. Warto tutaj podkreślić, że grupowanie było dokonane na podstawie wszystkich punktów (włącznie z tymi anotowanymi).






# Analiza po miesiącach

Ponieważ w poprzedniej części zadania grupowaliśmy dane pochodzące tylko z wybranych miesięcy, rozszerzymy teraz analizę dokonując klasteryzacji na danych z innych miesięcy. **Chcemy zbadać wpływ danych z ostatnich dwunastu miesięcy na wyniki grupowania.**

## Grupowanie k-średnich

### Dane z 01.09.2023

In [ ]:
SparkDataFrame_2023_9 = spark.sql("""
                        SELECT
                        *
                        FROM nasa_ym WHERE (Year == 2023) and (Month == 9)
                        order by lon, lat, Year, Month
                        """)

In [ ]:
df_2023_9 = SparkDataFrame_2023_9.toPandas()

In [ ]:
scaled_df_2023_9 = MinMaxScaling(df_2023_9, ['Evap', 'Rainf', 'AvgSurfT', 'Albedo', 'SoilM_100_200cm', 'GVEG', 'PotEvap', 'RootMoist', 'SoilT_10_40cm'])

In [ ]:
output_df_2023_9 = do_kmeans_and_return_df_with_cluster_column(df_2023_9, scaled_df_2023_9, 2, 123)

### Dane z 01.08.2023

In [ ]:
SparkDataFrame_2023_8 = spark.sql("""
                        SELECT
                        *
                        FROM nasa_ym WHERE (Year == 2023) and (Month == 8)
                        order by lon, lat, Year, Month
                        """)

In [ ]:
df_2023_8 = SparkDataFrame_2023_8.toPandas()

In [ ]:
scaled_df_2023_8 = MinMaxScaling(df_2023_8, ['Evap', 'Rainf', 'AvgSurfT', 'Albedo', 'SoilM_100_200cm', 'GVEG', 'PotEvap', 'RootMoist', 'SoilT_10_40cm'])

In [ ]:
output_df_2023_8 = do_kmeans_and_return_df_with_cluster_column(df_2023_8, scaled_df_2023_8, 2, 123)

### Dane z 01.06.2023

In [ ]:
SparkDataFrame_2023_6 = spark.sql("""
                        SELECT
                        *
                        FROM nasa_ym WHERE (Year == 2023) and (Month == 6)
                        order by lon, lat, Year, Month
                        """)

In [ ]:
df_2023_6 = SparkDataFrame_2023_6.toPandas()

In [ ]:
scaled_df_2023_6 = MinMaxScaling(df_2023_6, ['Evap', 'Rainf', 'AvgSurfT', 'Albedo', 'SoilM_100_200cm', 'GVEG', 'PotEvap', 'RootMoist', 'SoilT_10_40cm'])

In [ ]:
output_df_2023_6 = do_kmeans_and_return_df_with_cluster_column(df_2023_6, scaled_df_2023_6, 2, 123)

### Dane z 01.05.2023

In [ ]:
SparkDataFrame_2023_5 = spark.sql("""
                        SELECT
                        *
                        FROM nasa_ym WHERE (Year == 2023) and (Month == 5)
                        order by lon, lat, Year, Month
                        """)

In [ ]:
df_2023_5 = SparkDataFrame_2023_5.toPandas()

In [ ]:
scaled_df_2023_5 = MinMaxScaling(df_2023_5, ['Evap', 'Rainf', 'AvgSurfT', 'Albedo', 'SoilM_100_200cm', 'GVEG', 'PotEvap', 'RootMoist', 'SoilT_10_40cm'])

In [ ]:
output_df_2023_5 = do_kmeans_and_return_df_with_cluster_column(df_2023_5, scaled_df_2023_5, 2, 123)

### Dane z 01.04.2023

In [ ]:
SparkDataFrame_2023_4 = spark.sql("""
                        SELECT
                        *
                        FROM nasa_ym WHERE (Year == 2023) and (Month == 4)
                        order by lon, lat, Year, Month
                        """)

In [ ]:
df_2023_4 = SparkDataFrame_2023_4.toPandas()

In [ ]:
scaled_df_2023_4 = MinMaxScaling(df_2023_4, ['Evap', 'Rainf', 'AvgSurfT', 'Albedo', 'SoilM_100_200cm', 'GVEG', 'PotEvap', 'RootMoist', 'SoilT_10_40cm'])

In [ ]:
output_df_2023_4 = do_kmeans_and_return_df_with_cluster_column(df_2023_4, scaled_df_2023_4, 2, 123)

### Dane z 01.03.2023

In [ ]:
SparkDataFrame_2023_3 = spark.sql("""
                        SELECT
                        *
                        FROM nasa_ym WHERE (Year == 2023) and (Month == 3)
                        order by lon, lat, Year, Month
                        """)

In [ ]:
df_2023_3 = SparkDataFrame_2023_3.toPandas()

In [ ]:
scaled_df_2023_3 = MinMaxScaling(df_2023_3, ['Evap', 'Rainf', 'AvgSurfT', 'Albedo', 'SoilM_100_200cm', 'GVEG', 'PotEvap', 'RootMoist', 'SoilT_10_40cm'])

In [ ]:
output_df_2023_3 = do_kmeans_and_return_df_with_cluster_column(df_2023_3, scaled_df_2023_3, 2, 123)

### Dane z 01.02.2023

In [ ]:
SparkDataFrame_2023_2 = spark.sql("""
                        SELECT
                        *
                        FROM nasa_ym WHERE (Year == 2023) and (Month == 2)
                        order by lon, lat, Year, Month
                        """)

In [ ]:
df_2023_2 = SparkDataFrame_2023_2.toPandas()

In [ ]:
scaled_df_2023_2 = MinMaxScaling(df_2023_2, ['Evap', 'Rainf', 'AvgSurfT', 'Albedo', 'SoilM_100_200cm', 'GVEG', 'PotEvap', 'RootMoist', 'SoilT_10_40cm'])

In [ ]:
output_df_2023_2 = do_kmeans_and_return_df_with_cluster_column(df_2023_2, scaled_df_2023_2, 2, 123)

### Dane z 01.12.2022

In [ ]:
SparkDataFrame_2022_12 = spark.sql("""
                        SELECT
                        *
                        FROM nasa_ym WHERE (Year == 2022) and (Month == 12)
                        order by lon, lat, Year, Month
                        """)

In [ ]:
df_2022_12 = SparkDataFrame_2022_12.toPandas()

In [ ]:
scaled_df_2022_12 = MinMaxScaling(df_2022_12, ['Evap', 'Rainf', 'AvgSurfT', 'Albedo', 'SoilM_100_200cm', 'GVEG', 'PotEvap', 'RootMoist', 'SoilT_10_40cm'])

In [ ]:
output_df_2022_12 = do_kmeans_and_return_df_with_cluster_column(df_2022_12, scaled_df_2022_12, 2, 123)

### Dane z 01.11.2022

In [ ]:
SparkDataFrame_2022_11 = spark.sql("""
                        SELECT
                        *
                        FROM nasa_ym WHERE (Year == 2022) and (Month == 11)
                        order by lon, lat, Year, Month
                        """)

In [ ]:
df_2022_11 = SparkDataFrame_2022_11.toPandas()

In [ ]:
scaled_df_2022_11 = MinMaxScaling(df_2022_11, ['Evap', 'Rainf', 'AvgSurfT', 'Albedo', 'SoilM_100_200cm', 'GVEG', 'PotEvap', 'RootMoist', 'SoilT_10_40cm'])

In [ ]:
output_df_2022_11 = do_kmeans_and_return_df_with_cluster_column(df_2022_11, scaled_df_2022_11, 2, 123)

### Dane z 01.10.2022

In [ ]:
SparkDataFrame_2022_10 = spark.sql("""
                        SELECT
                        *
                        FROM nasa_ym WHERE (Year == 2022) and (Month == 10)
                        order by lon, lat, Year, Month
                        """)

In [ ]:
df_2022_10 = SparkDataFrame_2022_10.toPandas()

In [ ]:
scaled_df_2022_10 = MinMaxScaling(df_2022_10, ['Evap', 'Rainf', 'AvgSurfT', 'Albedo', 'SoilM_100_200cm', 'GVEG', 'PotEvap', 'RootMoist', 'SoilT_10_40cm'])

In [ ]:
output_df_2022_10 = do_kmeans_and_return_df_with_cluster_column(df_2022_10, scaled_df_2022_10, 2, 123)

## Przedstawienie klastrów na mapach

In [ ]:
display(IFrame("https://www.google.com/maps/embed?pb=!1m14!1m12!1m3!1d13982681.959428234!2d-98.66341902257437!3d38.39997874427714!2m3!1f0!2f0!3f0!3m2!1i1024!2i768!4f13.1!5e1!3m2!1spl!2spl!4v1703000232420!5m2!1spl!2spl", '800px', '500px'))

In [ ]:
colormap_cluster = get_colormap([0, max(output_df_2023_9.cluster.values)], ['yellow', 'darkgreen'])
output_notebook()
show(plot_map(df=output_df_2023_9, parameter_name='cluster', colormap=colormap_cluster, title="K-średnich dla danych z 01.09.2023", alpha=0.5))

In [ ]:
colormap_cluster = get_colormap([0, max(output_df_2023_8.cluster.values)], ['yellow', 'darkgreen'])
output_notebook()
show(plot_map(df=output_df_2023_8, parameter_name='cluster', colormap=colormap_cluster, title="K-średnich dla danych z 01.08.2023", alpha=0.5))

In [ ]:
colormap_cluster = get_colormap([0, max(output_df_2023_7.cluster.values)], ['yellow', 'darkgreen'])
output_notebook()
show(plot_map(df=output_df_2023_7, parameter_name='cluster', colormap=colormap_cluster, title="K-średnich dla danych z 01.07.2023", alpha=0.5))

In [ ]:
colormap_cluster = get_colormap([0, max(output_df_2023_6.cluster.values)], ['yellow', 'darkgreen'])
output_notebook()
show(plot_map(df=output_df_2023_6, parameter_name='cluster', colormap=colormap_cluster, title="K-średnich dla danych z 01.06.2023", alpha=0.5))

In [ ]:
colormap_cluster = get_colormap([0, max(output_df_2023_5.cluster.values)], ['yellow', 'darkgreen'])
output_notebook()
show(plot_map(df=output_df_2023_5, parameter_name='cluster', colormap=colormap_cluster, title="K-średnich dla danych z 01.05.2023", alpha=0.5))

In [ ]:
colormap_cluster = get_colormap([0, max(output_df_2023_4.cluster.values)], ['yellow', 'darkgreen'])
output_notebook()
show(plot_map(df=output_df_2023_4, parameter_name='cluster', colormap=colormap_cluster, title="K-średnich dla danych z 01.04.2023", alpha=0.5))

In [ ]:
colormap_cluster = get_colormap([0, max(output_df_2023_3.cluster.values)], ['yellow', 'darkgreen'])
output_notebook()
show(plot_map(df=output_df_2023_3, parameter_name='cluster', colormap=colormap_cluster, title="K-średnich dla danych z 01.03.2023", alpha=0.5))

In [ ]:
colormap_cluster = get_colormap([0, max(output_df_2023_2.cluster.values)], ['yellow', 'darkgreen'])
output_notebook()
show(plot_map(df=output_df_2023_2, parameter_name='cluster', colormap=colormap_cluster, title="K-średnich dla danych z 01.02.2023", alpha=0.5))

In [ ]:
colormap_cluster = get_colormap([0, max(output_df_2023_1.cluster.values)], ['yellow', 'darkgreen'])
output_notebook()
show(plot_map(df=output_df_2023_1, parameter_name='cluster', colormap=colormap_cluster, title="K-średnich dla danych z 01.01.2023", alpha=0.5))

In [ ]:
colormap_cluster = get_colormap([0, max(output_df_2022_12.cluster.values)], ['yellow', 'darkgreen'])
output_notebook()
show(plot_map(df=output_df_2022_12, parameter_name='cluster', colormap=colormap_cluster, title="K-średnich dla danych z 01.12.2022", alpha=0.5))

In [ ]:
colormap_cluster = get_colormap([0, max(output_df_2022_11.cluster.values)], ['yellow', 'darkgreen'])
output_notebook()
show(plot_map(df=output_df_2022_11, parameter_name='cluster', colormap=colormap_cluster, title="K-średnich dla danych z 01.11.2022", alpha=0.5))

In [ ]:
colormap_cluster = get_colormap([0, max(output_df_2022_10.cluster.values)], ['yellow', 'darkgreen'])
output_notebook()
show(plot_map(df=output_df_2022_10, parameter_name='cluster', colormap=colormap_cluster, title="K-średnich dla danych z 01.10.2022", alpha=0.5))

## Porównianie wyników z datasetem anotowanym

Porównajmy wyniki grupowań dla tych modeli, dla których graficzne przedstawienie klastrów w pewnym stopniu przypomina kształt pustyni.

In [ ]:
results = pd.DataFrame(
    {"lon": output_df_2023_9["lon"],
    "lat": output_df_2023_9["lat"],
    "cluster_2023_9": output_df_2023_9["cluster"].apply(lambda x: 0 if x==1 else 1),
    "cluster_2023_8": output_df_2023_8["cluster"].apply(lambda x: 0 if x==1 else 1),
    "cluster_2023_7": output_df_2023_7["cluster"],
    "cluster_2023_6": output_df_2023_6["cluster"],
    "cluster_2023_5": output_df_2023_5["cluster"],
    "cluster_2022_10": output_df_2022_10["cluster"].apply(lambda x: 0 if x==1 else 1)})

In [ ]:
results_with_an_pd = results.merge(NASA_sample_an[["lon", "lat", "pustynia_i_step"]], left_on=['lon','lat'], right_on = ['lon','lat'], how='inner')
results_with_an_pd.head()

In [ ]:
wyniki = pd.DataFrame({"Name": ["cluster_2023_9","cluster_2023_8","cluster_2023_7","cluster_2023_6","cluster_2023_5","cluster_2022_10"]})
wyniki["Accuracy [%]"] = wyniki["Name"].apply(lambda name: round(accuracy_score(results_with_an_pd["pustynia_i_step"],results_with_an_pd[name])*100, 2))
wyniki["Precision [%]"] = wyniki["Name"].apply(lambda name: round(precision_score(results_with_an_pd["pustynia_i_step"],results_with_an_pd[name])*100, 2))
wyniki["Recall [%]"] = wyniki["Name"].apply(lambda name: round(recall_score(results_with_an_pd["pustynia_i_step"],results_with_an_pd[name])*100, 2))
wyniki["F1-Score [%]"] = wyniki["Name"].apply(lambda name: round(f1_score(results_with_an_pd["pustynia_i_step"],results_with_an_pd[name])*100, 2))
wyniki.sort_values(by=['Accuracy [%]'], ascending=False)

Jak widzimy model wytrenowany na danych z lipca 2023 poradził sobie najlepiej z datasetem anotowanym.

# Wykorzystanie najlepszego modelu do grupowania danych z innych miesięcy

Celem tej analizy jest wykorzystanie najlepszego modelu, którym jest model wytrenowany na danych z lipca 2023, do predykcji danych z innych miesięcy.

In [ ]:
kmeans = KMeans(n_clusters = 2, init='k-means++', random_state=123)
kmeans.fit(scaled_df_2023_7)

In [ ]:
results_with_best_model = pd.DataFrame(
    {"lon": output_df_2023_9["lon"],
    "lat": output_df_2023_9["lat"],
     "cluster_2023_9": kmeans.predict(scaled_df_2023_9),
     "cluster_2023_8": kmeans.predict(scaled_df_2023_8),
     "cluster_2023_7": kmeans.predict(scaled_df_2023_7),
     "cluster_2023_6": kmeans.predict(scaled_df_2023_6),
     "cluster_2023_5": kmeans.predict(scaled_df_2023_5),
     "cluster_2023_4": kmeans.predict(scaled_df_2023_4),
     "cluster_2023_3": kmeans.predict(scaled_df_2023_3),
     "cluster_2023_2": kmeans.predict(scaled_df_2023_2),
     "cluster_2023_1": kmeans.predict(scaled_df_2023_1),
     "cluster_2022_12": kmeans.predict(scaled_df_2022_12),
     "cluster_2022_11": kmeans.predict(scaled_df_2022_11),
     "cluster_2022_10": kmeans.predict(scaled_df_2022_10)
     })
results_with_best_model

In [ ]:
results_with_best_model_with_an_pd = results_with_best_model.merge(NASA_sample_an[["lon", "lat", "pustynia_i_step"]], left_on=['lon','lat'], right_on = ['lon','lat'], how='inner')
results_with_best_model_with_an_pd.head()

In [ ]:
wyniki = pd.DataFrame({"Name": ["cluster_2023_9","cluster_2023_8","cluster_2023_7","cluster_2023_6","cluster_2023_5","cluster_2023_4","cluster_2023_3","cluster_2023_2","cluster_2023_1","cluster_2022_12","cluster_2022_11","cluster_2022_10"]})
wyniki["Accuracy [%]"] = wyniki["Name"].apply(lambda name: round(accuracy_score(results_with_best_model_with_an_pd["pustynia_i_step"],results_with_best_model_with_an_pd[name])*100, 2))
wyniki["Precision [%]"] = wyniki["Name"].apply(lambda name: round(precision_score(results_with_best_model_with_an_pd["pustynia_i_step"],results_with_best_model_with_an_pd[name])*100, 2))
wyniki["Recall [%]"] = wyniki["Name"].apply(lambda name: round(recall_score(results_with_best_model_with_an_pd["pustynia_i_step"],results_with_best_model_with_an_pd[name])*100, 2))
wyniki["F1-Score [%]"] = wyniki["Name"].apply(lambda name: round(f1_score(results_with_best_model_with_an_pd["pustynia_i_step"],results_with_best_model_with_an_pd[name])*100, 2))
wyniki.sort_values(by=['Accuracy [%]'], ascending=False)

Widać, że predykcja danych z ostatnich miesięcy za pomocą modelu wytrenowanego na danych z lipca 2023 jest najlepsza dla danych z 01.07.2023.

# Grupowanie na podstawie pojedynczej kolumny

## Implementacja algorytmu

In [ ]:
scaled_df_2023_7_Evap = MinMaxScaling(df_2023_7, ['Evap'])
output_df_2023_7_Evap = do_kmeans_and_return_df_with_cluster_column(df_2023_7[["lon", "lat", "Evap"]], scaled_df_2023_7_Evap, 2, 123)

In [ ]:
scaled_df_2023_7_Rainf = MinMaxScaling(df_2023_7, ['Rainf'])
output_df_2023_7_Rainf = do_kmeans_and_return_df_with_cluster_column(df_2023_7[["lon", "lat", "Rainf"]], scaled_df_2023_7_Rainf, 2, 123)

In [ ]:
scaled_df_2023_7_AvgSurfT = MinMaxScaling(df_2023_7, ['AvgSurfT'])
output_df_2023_7_AvgSurfT = do_kmeans_and_return_df_with_cluster_column(df_2023_7[["lon", "lat", "AvgSurfT"]], scaled_df_2023_7_AvgSurfT, 2, 123)

In [ ]:
scaled_df_2023_7_Albedo = MinMaxScaling(df_2023_7, ['Albedo'])
output_df_2023_7_Albedo = do_kmeans_and_return_df_with_cluster_column(df_2023_7[["lon", "lat", "Albedo"]], scaled_df_2023_7_Albedo, 2, 123)

In [ ]:
scaled_df_2023_7_SoilM_100_200cm = MinMaxScaling(df_2023_7, ['SoilM_100_200cm'])
output_df_2023_7_SoilM_100_200cm = do_kmeans_and_return_df_with_cluster_column(df_2023_7[["lon", "lat", "SoilM_100_200cm"]], scaled_df_2023_7_SoilM_100_200cm, 2, 123)

In [ ]:
scaled_df_2023_7_GVEG = MinMaxScaling(df_2023_7, ['GVEG'])
output_df_2023_7_GVEG = do_kmeans_and_return_df_with_cluster_column(df_2023_7[["lon", "lat", "GVEG"]], scaled_df_2023_7_GVEG, 2, 123)

In [ ]:
scaled_df_2023_7_PotEvap = MinMaxScaling(df_2023_7, ['PotEvap'])
output_df_2023_7_PotEvap = do_kmeans_and_return_df_with_cluster_column(df_2023_7[["lon", "lat", "PotEvap"]], scaled_df_2023_7_PotEvap, 2, 123)

In [ ]:
scaled_df_2023_7_RootMoist = MinMaxScaling(df_2023_7, ['RootMoist'])
output_df_2023_7_RootMoist = do_kmeans_and_return_df_with_cluster_column(df_2023_7[["lon", "lat", "RootMoist"]], scaled_df_2023_7_RootMoist, 2, 123)

In [ ]:
scaled_df_2023_7_SoilT_10_40cm = MinMaxScaling(df_2023_7, ['SoilT_10_40cm'])
output_df_2023_7_SoilT_10_40cm = do_kmeans_and_return_df_with_cluster_column(df_2023_7[["lon", "lat", "SoilT_10_40cm"]], scaled_df_2023_7_SoilT_10_40cm, 2, 123)

## Przedstawienie wyników na mapach

In [ ]:
colormap_cluster = get_colormap([0, max(output_df_2023_7_Evap.cluster.values)], ['yellow', 'darkgreen'])
output_notebook()
show(plot_map(df=output_df_2023_7_Evap, parameter_name='cluster', colormap=colormap_cluster, title="K-średnich dla danych z 01.07.2023 - Evap", alpha=0.5))

In [ ]:
colormap_cluster = get_colormap([0, max(output_df_2023_7_Rainf.cluster.values)], ['yellow', 'darkgreen'])
output_notebook()
show(plot_map(df=output_df_2023_7_Rainf, parameter_name='cluster', colormap=colormap_cluster, title="K-średnich dla danych z 01.07.2023 - Rainf", alpha=0.5))

In [ ]:
colormap_cluster = get_colormap([0, max(output_df_2023_7_AvgSurfT.cluster.values)], ['yellow', 'darkgreen'])
output_notebook()
show(plot_map(df=output_df_2023_7_AvgSurfT, parameter_name='cluster', colormap=colormap_cluster, title="K-średnich dla danych z 01.07.2023 - AvgSurfT", alpha=0.5))

In [ ]:
colormap_cluster = get_colormap([0, max(output_df_2023_7_Albedo.cluster.values)], ['yellow', 'darkgreen'])
output_notebook()
show(plot_map(df=output_df_2023_7_Albedo, parameter_name='cluster', colormap=colormap_cluster, title="K-średnich dla danych z 01.07.2023 - Albedo", alpha=0.5))

In [ ]:
colormap_cluster = get_colormap([0, max(output_df_2023_7_SoilM_100_200cm.cluster.values)], ['yellow', 'darkgreen'])
output_notebook()
show(plot_map(df=output_df_2023_7_SoilM_100_200cm, parameter_name='cluster', colormap=colormap_cluster, title="K-średnich dla danych z 01.07.2023 - SoilM_100_200cm", alpha=0.5))

In [ ]:
colormap_cluster = get_colormap([0, max(output_df_2023_7_GVEG.cluster.values)], ['yellow', 'darkgreen'])
output_notebook()
show(plot_map(df=output_df_2023_7_GVEG, parameter_name='cluster', colormap=colormap_cluster, title="K-średnich dla danych z 01.07.2023 - GVEG", alpha=0.5))

In [ ]:
colormap_cluster = get_colormap([0, max(output_df_2023_7_PotEvap.cluster.values)], ['yellow', 'darkgreen'])
output_notebook()
show(plot_map(df=output_df_2023_7_PotEvap, parameter_name='cluster', colormap=colormap_cluster, title="K-średnich dla danych z 01.07.2023 - PotEvap", alpha=0.5))

In [ ]:
colormap_cluster = get_colormap([0, max(output_df_2023_7_RootMoist.cluster.values)], ['yellow', 'darkgreen'])
output_notebook()
show(plot_map(df=output_df_2023_7_RootMoist, parameter_name='cluster', colormap=colormap_cluster, title="K-średnich dla danych z 01.07.2023 - RootMoist", alpha=0.5))

In [ ]:
colormap_cluster = get_colormap([0, max(output_df_2023_7_SoilT_10_40cm.cluster.values)], ['yellow', 'darkgreen'])
output_notebook()
show(plot_map(df=output_df_2023_7_SoilT_10_40cm, parameter_name='cluster', colormap=colormap_cluster, title="K-średnich dla danych z 01.07.2023 - SoilT_10_40cm", alpha=0.5))

## Grupowanie na podstawie wszystkich kombinacji par kolumn.

In [ ]:
lista_kombinacji = list(combinations(['Evap', 'Rainf', 'AvgSurfT', 'Albedo', 'SoilM_100_200cm', 'GVEG', 'PotEvap', 'RootMoist', 'SoilT_10_40cm'],2))

In [ ]:
for para in lista_kombinacji:
  col1, col2 = para
  scaled_df_2023_7_mix = MinMaxScaling(df_2023_7, [col1, col2])
  output_df_2023_7_mix = do_kmeans_and_return_df_with_cluster_column(df_2023_7[["lon", "lat", col1, col2]], scaled_df_2023_7_mix, 2, 123)
  colormap_cluster = get_colormap([0, max(output_df_2023_7_mix.cluster.values)], ['yellow', 'darkgreen'])
  output_notebook()
  show(plot_map(df=output_df_2023_7_mix, parameter_name='cluster', colormap=colormap_cluster, title= "".join(["K-średnich dla danych z 01.07.2023 - ", col1," i ", col2]), alpha=0.5))

In [ ]:
#Zapisanie pliku jako html
#!jupyter nbconvert --to html "/content/Grupowanie_k_meanss.ipynb"